## Task1: Cryptarithmetic puzzle 

First the program needs to ask a puzzle.

In [55]:
from simpleai.search import CspProblem, backtrack
puzzle = input("Give a cryptarithmetic puzzle.")

### Validating the input 
We have gotten a string, but it might not be in a correct form (... + ... = ...). We will have check if the string is usable. The input needs has the following validation rules:
1. There is an operator (+, -, * or /)
2. There is an = sign
3. There are 3 words which are split up by the operator and the = sign


In [56]:
import re  
# The puzzle has only one operator
def validate_operator(puzzle : str):
    pattern = r'^[^+\-*/]*[+\-*/][^+\-*/]*$'
    if re.match(pattern, puzzle):
        return True
    else:
        raise Exception("""The cryptarithmetic puzzle is not in the correct form. Use one operator (+, -, *, /) to define your calculation.""")

# the puzzle has an only one = sign
def validate_equal_sign(puzzle : str):
    if puzzle.count('=') == 1:
        return True
    else: 
        raise Exception("""The cryptarithmetic puzzle is not in the correct form. Use one "=" sign.""")
    
# The puzzle has 3 words which are split up by the operator and the = sign
def validate_3words(puzzle : str):
    pattern = r'^[a-zA-Z]+\s*[+\-*/]\s*[a-zA-Z]+\s*=\s*[a-zA-Z]+$'

    modified_string = re.sub(r'\s', '', puzzle)
    if re.match(pattern, modified_string) :
        return True
    else:
        raise Exception("""The cryptarithmetic puzzle is not in the correct form. The correct form is: "ai + is = fun", you will need 3 words.""")

def validate_puzzle(puzzle : str):
    validate_operator(puzzle)
    validate_equal_sign(puzzle)
    validate_3words(puzzle)


Apply the validation rules to the puzzle.

In [57]:
validate_puzzle(puzzle)

### Getting the operator and the words
The next step is to define a few things:
1. The operator ( +, -, *, / )
2. We need the 3 words, the first 2 words in the calculation and the result word.


In [58]:
def find_operator(puzzle):
     if "+" in puzzle:
          operator = "+"
     elif "-" in puzzle:
          operator = "-"
     elif "*" in puzzle:
          operator = "*"
     else:
           operator = "/"
     return operator

Find the operator.

In [59]:
operator = find_operator(puzzle)
print(operator)

+


For the words we slice the given puzzle. Because we defined the correct form of the puzzle as 'word1 + word2 = result', between the 2 first there is an operator and in between the 2 last words there is an equal sign. Because we know this, we can slice the puzzle by finding the operator and the equal sign. After slicing the puzzle we remove the spaces and we capitalize the letters.

In [60]:
word_1 = puzzle[:puzzle.index(operator)].replace(" ","").upper()
word_2 = puzzle[puzzle.index(operator)+1:puzzle.index("=")].replace(" ","").upper()
word_result = puzzle[puzzle.index("=")+1:].replace(" ","").upper()

print(f"{word_1} {operator} {word_2} = {word_result}")


TO + GO = OUT


### Unique letters and the possible numbers
Next we want to create a tuple of all the unique letters. 

In [61]:

def unique_letters(word_1 : str, word_2 : str, word_result : str):
    letters = []
    for word in (word_1, word_2, word_result):
        for char in word:
            if char not in letters:
                letters.append(char)
    return tuple(letters)

Use the function to get the unique letters.

In [62]:
letters = unique_letters(word_1, word_2, word_result)
print(letters)

('T', 'O', 'G', 'U')


Also a dictionairy where each possible number is appointed to a letter. Of course the first letters of the words cannot have 0 as a value.

In [68]:
def possible_values(letters : tuple):
    domains = {}

    for letter in letters:
        if letter in [word_1[0], word_2[0], word_result[0]]:
            domains[letter] = list(range(1, 10))
        else:
            domains[letter] = list(range(0, 10))
    return domains



Use the function to create the dictionairy with possible values.

In [69]:
domains = possible_values(letters)
print(domains)

{'T': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'O': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'G': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'U': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


### Constraints
We need 2 constraints, one constraint to make sure that there are no duplicates. The other calculates the result of the operation and checks this with the given result.

In [64]:
# constraint 1
def constraint_unique(variables, values):
    return len(values) == len(set(values))

# helper function for constraint 2
def word_as_number(word : str, values, variables):
    number = ""
    for letter in word:
        number += str(values[variables.index(letter)])
    return int(number)

#constraint 2
def constraint_calculation(variables, values):

    number_1 = word_as_number(word_1, values, variables)
    number_2 = word_as_number(word_2, values, variables)
    result = word_as_number(word_result, values, variables)

    if operator ==  "+":
        return (number_1 + number_2) == result
    elif operator == "-":
        return result == (number_1 - number_2) == result
    elif operator == "*":
        return (number_1 * number_2) == result
    else:
        return (number_1 / number_2) ==result

The constraints are applied to all the letters.

In [65]:
constraints = [
    (letters, constraint_unique),
    (letters, constraint_calculation),
]

## Backtracking

Now we can define the problem and backtrack until we have a solution.

In [66]:
problem = CspProblem(letters, domains, constraints)

output = backtrack(problem)
print('Solutions:', output)

Solutions: {'T': 2, 'O': 1, 'G': 8, 'U': 0}


### Output
To print the output I create another function to find the numberical value for each word.

In [67]:
def number_result(word : str,solutions : dict):
    number = ""
    for letter in word:
        number += str(solutions[letter])
    return int(number)

print(f"{word_1} {operator} {word_2} = {word_result}")
print(f"""{number_result(word_1, output)} {operator} {number_result(word_2, output)} = {number_result(word_result, output)}""")

TO + GO = OUT
21 + 81 = 102
